In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
from pathlib import Path
import cv2
from functools import partial

from watch_recognition.data_preprocessing import load_keypoints_data_as_kp
from watch_recognition.augmentations import set_shapes

In [ ]:
IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 100
NUM_KEYPOINTS = 4 * 2  # 24 pairs each having x and y coordinates

In [ ]:
def get_model():
    # Load the pre-trained weights of MobileNetV2 and freeze the weights
    backbone = tf.keras.applications.EfficientNetB0(
        weights="imagenet", include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3)
    )
    backbone.trainable = True

    inputs = layers.Input((IMG_SIZE, IMG_SIZE, 3))
    x = keras.applications.efficientnet.preprocess_input(inputs)
    x = backbone(x)
    x = layers.Conv2D(
        1024, kernel_size=5, strides=1, activation="relu"
    )(x)
    # x = layers.Dropout(0.3)(x)
    x = layers.Flatten()(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dense(NUM_KEYPOINTS, activation='sigmoid')(x)
    # x = layers.SeparableConv2D(
    #     NUM_KEYPOINTS, kernel_size=5, strides=1, activation="relu"
    # )(x)
    # outputs = layers.SeparableConv2D(
    #     NUM_KEYPOINTS, kernel_size=3, strides=1, activation="sigmoid"
    # )(x)

    return keras.Model(inputs, x, name="keypoint_detector")

In [ ]:
get_model().summary()

In [ ]:
X, y = load_keypoints_data_as_kp(
    Path("../download_data/keypoints/train"),
)
X.shape, y.shape

In [ ]:
X_val, y_val = load_keypoints_data_as_kp(
    Path("../download_data/keypoints/validation"),
)
y_val = y_val[:, :2].reshape(-1, NUM_KEYPOINTS) / IMG_SIZE
X_val.shape, y_val.shape

In [ ]:

dataset = tf.data.Dataset.from_tensor_slices((X, y))

In [ ]:
def process_data(image, kp):
    image = tf.cast(image, tf.float32)
    kp = kp /IMG_SIZE
    kp = tf.reshape(kp[:, :2], (-1, NUM_KEYPOINTS))
    return image, kp

train_ds = dataset.map(process_data)

In [ ]:
img, kp = next(iter(train_ds))

In [ ]:
kp.numpy().reshape((-1,2))

In [ ]:

def vis_keypoints(image, keypoints, color=(0, 255, 0)):
    image = image.copy().astype('uint8')
    diameter = int(np.mean([image.shape[0], image.shape[1]])/50)
    keypoints = keypoints * IMG_SIZE
    for kp in keypoints:
        x, y, = kp[0], kp[1]
        if  2 < x < image.shape[0]-2 and 2 < y <  image.shape[1]-2 :
            x_int = int(x)
            y_int = int(y)
            cv2.circle(image, (x_int, y_int), diameter, color, -1)
        else:
            print(f"kp {x},{y} ignored")

    plt.figure(figsize=(8, 8))
    plt.axis('off')
    plt.imshow(image)
vis_keypoints(img.numpy(), kp.numpy().reshape((-1,2)))

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

ds_alb = train_ds.map(
    partial(set_shapes, img_shape=(224, 224, 3), target_shape=(1, NUM_KEYPOINTS)),
    num_parallel_calls=AUTOTUNE).shuffle(8*BATCH_SIZE).batch(BATCH_SIZE).prefetch(AUTOTUNE)
ds_alb

In [ ]:
model = get_model()
model.compile(loss="mse", optimizer=keras.optimizers.Adam())
model.fit(ds_alb, validation_data=(X_val, y_val), epochs=EPOCHS)

In [ ]:
sample_val_images, sample_val_keypoints = next(iter(ds_alb))
sample_val_images = sample_val_images[:1].numpy()
sample_val_keypoints = sample_val_keypoints[0].numpy().reshape(-1, 2)# * IMG_SIZE
predictions = model.predict(sample_val_images).reshape(-1, 2)# * IMG_SIZE

In [ ]:
# Ground-truth
vis_keypoints(sample_val_images[0], sample_val_keypoints)

In [ ]:
# Predictions
vis_keypoints(sample_val_images[0], predictions)

In [ ]:
predictions